# Kaggle competition
# Web Traffic Time Series Forecasting
# https://www.kaggle.com/c/web-traffic-time-series-forecasting

It uses an PROPHET model to predict pages visualisation
it takes the key_2 file from the Kaggle competition page with the pages,
and dates for each page, to predict, and use the train_2 file as
training dataset


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas
import time
from pandas import datetime
from fbprophet import Prophet
import numpy as np

# Set pandas not to cut long strings
pandas.set_option("display.max_colwidth",10000)


Import data

In [ ]:
# Import keys for output dataframe to submit
dataframe_keys = pandas.read_csv('key_2.csv')

# Import train data for output dataframe to submit
dataframe_train = pandas.read_csv('train_2.csv')
dataframe_train = dataframe_train.fillna(0)
print(dataframe_keys.head(3))
print(dataframe_train.head(3))

Manimulate keys dataframe to prepare for predictions and extract dates

In [ ]:
# Prepare a column of zeros as initial prediction of the visits for each key 
dataframe_keys['Visits'] = np.zeros(dataframe_keys.shape[0])

# Scrapping the date from the page column in the keys dataframe
dataframe_keys['Page'] = dataframe_keys['Page'].apply(lambda x: x.split('-')[:-2])
dataframe_keys['Page'] = dataframe_keys['Page'].str.join('-')
dataframe_keys['Page'] = dataframe_keys['Page'].apply(lambda x: x.split('_')[:-1])
dataframe_keys['Page'] = dataframe_keys['Page'].str.join('_')

# Create a list called "indices" with all the pages to be predicted
# And the corresponding indices in the dataframe
indices = dataframe_keys.groupby('Page').apply(lambda x: x.index.tolist())
print(dataframe_keys.head())
print(indices[:2])


# Loops over all pages to make predictions

In [ ]:
def write_predictions(save_pages,ipage,startg,start,dataframe_keys):
    if ipage % save_pages == 0:
        print('Predicted %1.f pages:' % ipage)
        deltatg = time.time()-startg
        print('Total time elapsed: %.2f (sec)'% deltatg)
        deltat = time.time()-start
        print('Time elapsed last %d pages: %.2f (sec)'% (save_pages,deltat))
        print('Writing predictions')
        header_to_print = ['Id','Visits']
        dataframe_keys.to_csv('output.csv', columns=header_to_print, index=False)
        print('Predictions written')
        print()
        start = time.time()
    return start


In [ ]:
# Set every how many pages the results are saved on disk
save_pages = 10


# Calculate the global time from beginning of predictions
startg = time.time()
# Calculate how much time it takes to predict
# save_pages number of pages
start = time.time()


# Make predictions only on the first 100 pages
# to check if the loop works
for ipage in range(21):
#for ipage in range(len(dataframe_train)):
    page_to_predict = dataframe_train.iloc[[ipage]]
    page_name = page_to_predict.Page.values[0]

    indices_page = indices.loc[page_name]
    page_to_predict = page_to_predict.drop(['Page'], axis=1)
    # prepare dataframe for Prophet
    df = pandas.DataFrame()
    df['ds'] = list(page_to_predict.columns)
    df['y']  = page_to_predict.values[0]
    df['ds'] = pandas.to_datetime(df['ds'])
    
    # Make predictions (initialize predictions with mean)
    predictions = np.mean(df.y.mean())*np.ones(len(indices_page))
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        # Set the model to be Prophet
        m = Prophet(daily_seasonality=True)
        m.fit(df)
        future = m.make_future_dataframe(periods=len(indices_page))
        forecast = m.predict(future)
        predictions = forecast.yhat.values[-len(indices_page):]

    print('Predicted page # %1.f:' % ipage,page_name)
    for ipred in range(len(indices_page)):
        dataframe_keys.set_value(indices_page[ipred],'Visits',predictions[ipred])

    # writes the predictions after save_pages number of pages
    # returns the start time to calculate how long it takes
    # to predict save_pages number of pages
    start = write_predictions(save_pages,ipage,startg,start,dataframe_keys)
        
      
print('Save last chunk of predictions')
header_to_print = ['Id','Visits']
dataframe_keys.to_csv('output.csv', columns=header_to_print, index=False)
print('Predictions written')

